# 6.7 Final Project: Tableau Dashboards and Final Analysis

## About this notebook

This notebook contains a final iteration of all the techniques and methodologies practiced in this achievement. This iteration introduces suplementary datasets to delve deeper into the observations made throughout the case study. The datasets include an index of gun law strength and gun violence by Everytown, a non-governmental research organization in the United States focused on gun violence prevention, and population densities at state and county granularity. 

The objective of this notebook is to re-perform exploratory analysis, geographic visualization, linear regression and cluster analysis. 

## Part 1

### This script contains

##### 1. Import of libraries and data
##### 2. Data cleaning
##### 3. Data wrangling

####  1. Import of libraries and data

In [9]:
import pandas as pd
import numpy as np
import os
import json
import geojson

In [11]:
#Ensuring matplotlib display 
%matplotlib inline

In [12]:
#Importing files
path = r'C:/Users/C SaiVishwanath/Documents/CF/Data Immersion/Achievement 6'

In [102]:
df = pd.read_csv(os.path.join(path, '02_Data', 'Prepared', '130425_mass_shoot_merged_derived_ver3.csv'), index_col = False, encoding='latin1')
df_gunindex = pd.read_csv(os.path.join(path, '02_Data', 'Original', 'Gun Safety Indicators.csv'), sep= ';', index_col = False)
df_popdenstate = pd.read_csv(os.path.join(path, '02_Data', 'Original', 'Population-Density By State.csv'), index_col = False)
df_popdencounty = pd.read_csv(os.path.join(path, '02_Data', 'Original', 'counties_pop density.csv'), index_col = False) 
county_geo = r'C:/Users/C SaiVishwanath/Documents/CF/Data Immersion/Achievement 6/02_Data/Original/counties.geojson'

#### 2. Data cleaning

#### Main dataframe

In [19]:
df.head()

,Incident_ID,Date,Year,Month,State,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,...,State_ID,State_y,County,Lat,Long,Population,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested
0,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,...,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No
1,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,...,FL,Florida,Manatee,27.5435,-82.5608,0.0,3,30,6,No
2,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,...,TX,Texas,Hall,34.7268,-100.5416,0.0,3,30,6,No
3,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,...,MO,Missouri,Scotland,40.4613,-92.1704,0.0,3,30,6,No
4,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,...,MI,Michigan,Macomb,42.8956,-82.7689,0.0,3,30,6,No


In [21]:
#Missing values check
df.isnull().sum()

Incident_ID             0
Date                    0
Year                    0
Month                   0
State                   0
City                    0
Victims_Killed          0
Victims_Injured         0
Suspects_Killed         0
Suspects_Injured        0
Suspects_Arrested       0
Year_Month_State        0
Handguns_Sold           0
Long_Guns_Sold          0
Total_Guns_Sold         0
State_ID                0
State_y                 0
County                  0
Lat                     0
Long                    0
Population              0
Shootings_County        0
Shootings_State         0
Total_Harmed_Victims    0
Any_Suspect_Arrested    0
dtype: int64

In [23]:
#Duplicates check
dups = df.duplicated()
dups.sum()

16157

In [25]:
pd.set_option('display.max_columns', None)

In [27]:
df[df.duplicated()]

,Incident_ID,Date,Year,Month,State,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested
39462,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No
39463,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No
39464,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No
39465,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No
39466,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64516,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes
64517,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes
64518,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes
64519,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes


In [29]:
#Dropping duplicates
df = df.drop_duplicates()

In [31]:
df[df.duplicated()]

,Incident_ID,Date,Year,Month,State,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested


In [33]:
df.shape

(48371, 25)

In [35]:
df.dtypes

Incident_ID               int64
Date                     object
Year                      int64
Month                     int64
State                    object
City                     object
Victims_Killed            int64
Victims_Injured           int64
Suspects_Killed           int64
Suspects_Injured          int64
Suspects_Arrested         int64
Year_Month_State         object
Handguns_Sold           float64
Long_Guns_Sold          float64
Total_Guns_Sold         float64
State_ID                 object
State_y                  object
County                   object
Lat                     float64
Long                    float64
Population              float64
Shootings_County          int64
Shootings_State           int64
Total_Harmed_Victims      int64
Any_Suspect_Arrested     object
dtype: object

In [37]:
df.describe()

,Incident_ID,Year,Month,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Lat,Long,Population,Shootings_County,Shootings_State,Total_Harmed_Victims
count,4.837100e+04,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,48371.000000,4.837100e+04,48371.000000,48371.000000,48371.000000
mean,2.134064e+06,2021.257179,6.692646,0.965144,4.167146,0.064109,0.047404,0.683571,29106.847305,18007.645201,47114.464845,38.287983,-89.950253,2.297070e+05,66.978934,455.253065,5.132290
std,7.222183e+05,2.459768,3.039073,1.323983,3.708707,0.252267,0.239318,1.142419,23419.081251,12159.116460,34371.181760,4.380785,10.917209,1.368722e+06,99.381771,318.305968,4.039275
min,2.720160e+05,2015.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.977800,-170.274300,0.000000e+00,1.000000,1.000000,0.000000
25%,1.713668e+06,2020.000000,5.000000,0.000000,3.000000,0.000000,0.000000,0.000000,11740.000000,9880.000000,22656.000000,35.316900,-94.379900,0.000000e+00,4.000000,198.000000,4.000000
50%,2.346406e+06,2022.000000,7.000000,1.000000,4.000000,0.000000,0.000000,0.000000,23085.000000,15348.000000,37597.000000,39.044800,-88.166600,1.554700e+04,20.000000,387.000000,4.000000
75%,2.691769e+06,2023.000000,9.000000,1.000000,5.000000,0.000000,0.000000,1.000000,37859.000000,23422.000000,61953.000000,41.031800,-82.985500,5.542700e+04,90.000000,669.000000,5.000000
max,3.181158e+06,2025.000000,12.000000,60.000000,439.000000,3.000000,5.000000,14.000000,171600.000000,102546.000000,248724.000000,68.344500,-65.773300,3.129083e+07,483.000000,1303.000000,499.000000


#### Gun Index

In [40]:
df_gunindex.head()

,State,Category,Gun_Law_Strength,Gun_Violence_Rate
0,California,National leader,"90,5",8
1,Massachusetts,National leader,"86,5","3,7"
2,Illinois,National leader,85,"13,5"
3,New York,National leader,85,"4,7"
4,Connecticut,National leader,"81,5","6,2"


In [42]:
df_gunindex.shape

(50, 4)

In [44]:
#Missing values check
df_gunindex.isnull().sum()

State                0
Category             0
Gun_Law_Strength     0
Gun_Violence_Rate    0
dtype: int64

In [46]:
#Duplicates check
dups_gindex = df_gunindex.duplicated()
dups_gindex.sum()

0

In [48]:
df_gunindex.describe()

,State,Category,Gun_Law_Strength,Gun_Violence_Rate
count,50,50,50,50
unique,50,5,40,46
top,California,National failures,14,"21,5"
freq,1,14,3,2


In [50]:
df_gunindex.dtypes

State                object
Category             object
Gun_Law_Strength     object
Gun_Violence_Rate    object
dtype: object

In [52]:
#Converting data types in indexes
#Changing comma for period
cols = ['Gun_Law_Strength', 'Gun_Violence_Rate']

df_gunindex[cols] = df_gunindex[cols].apply(lambda x: x.astype(str).str.replace(',', '.', regex=False))

In [54]:
df_gunindex.head()

,State,Category,Gun_Law_Strength,Gun_Violence_Rate
0,California,National leader,90.5,8
1,Massachusetts,National leader,86.5,3.7
2,Illinois,National leader,85,13.5
3,New York,National leader,85,4.7
4,Connecticut,National leader,81.5,6.2


In [56]:
#Changing data types
df_gunindex[cols] = df_gunindex[cols].astype(float)

In [58]:
df_gunindex.dtypes

State                 object
Category              object
Gun_Law_Strength     float64
Gun_Violence_Rate    float64
dtype: object

In [60]:
df_gunindex.describe()

,Gun_Law_Strength,Gun_Violence_Rate
count,50.00000,50.000000
mean,33.46000,15.212000
std,28.89369,6.144573
min,2.50000,3.700000
25%,9.50000,12.075000
50%,21.00000,14.500000
75%,58.75000,18.575000
max,90.50000,29.400000


#### Population density by state

In [63]:
df_popdenstate.head()

,GEO.id,GEO.id2,GEO.display-label,Density per square mile of land area
0,0400000US01,1,Alabama,94.4
1,0400000US02,2,Alaska,1.2
2,0400000US04,4,Arizona,56.3
3,0400000US05,5,Arkansas,56.0
4,0400000US06,6,California,239.1


In [65]:
#Missing values check
df_popdenstate.isnull().sum()

GEO.id                                  0
GEO.id2                                 0
GEO.display-label                       0
Density per square mile of land area    0
dtype: int64

In [66]:
#Duplicates check
df_popdenstate = df_popdenstate.duplicated()
df_popdenstate.sum()

0

In [69]:
df_popdenstate.dtypes

dtype('bool')

In [77]:
#Changing data types
cols_obj_denstate = ['GEO.id', 'GEO.id2', 'GEO.display-label']

df_popdenstate[cols_obj_denstate] = df_popdenstate[cols_obj_denstate].astype(str)

In [79]:
print(df_popdenstate.columns.tolist())

['GEO.id', 'GEO.id2', 'GEO.display-label', 'Density per square mile of land area']


In [81]:
df_popdenstate.columns = df_popdenstate.columns.str.strip()

In [83]:
#Changing data types
cols_obj_denstate = ['GEO.id', 'GEO.id2', 'GEO.display-label']

df_popdenstate[cols_obj_denstate] = df_popdenstate[cols_obj_denstate].astype(str)

In [85]:
cols_float_denstate = ['Density per square mile of land area']

df_popdenstate[cols_float_denstate] = df_popdenstate[cols_float_denstate].astype(float)

In [87]:
df_popdenstate.dtypes

GEO.id                                   object
GEO.id2                                  object
GEO.display-label                        object
Density per square mile of land area    float64
dtype: object

In [89]:
df_popdenstate.describe()

,Density per square mile of land area
count,52.000000
mean,397.938462
std,1367.272141
min,1.200000
25%,47.150000
50%,103.050000
75%,233.100000
max,9856.500000


#### Population density by county

In [92]:
df_popdencounty.head()

,County,State,FIPS Code,Population,Area,Density
0,Autauga County,Alabama,01001,61464,594,103
1,Baldwin County,Alabama,01003,261608,1589,165
2,Barbour County,Alabama,01005,24358,884,28
3,Bibb County,Alabama,01007,22258,622,36
4,Blount County,Alabama,01009,60163,644,93


In [94]:
#Missing values check
df_popdencounty.isnull().sum()

County        0
State         0
FIPS Code     0
Population    0
Area          0
Density       0
dtype: int64

In [96]:
#Duplicates check
df_popdencounty = df_popdenstate.duplicated()
df_popdencounty.sum()

0

In [98]:
#Checking data types
df_popdencounty.dtypes

dtype('bool')

In [104]:
print(df_popdencounty.columns.tolist())

['County', 'State', 'FIPS Code', 'Population', 'Area', 'Density']


In [106]:
#Changing data types: object
cols_obj_dencounty = ['County', 'State', 'FIPS Code']

df_popdencounty[cols_obj_dencounty] = df_popdencounty[cols_obj_dencounty].astype(str)

In [108]:
#Changing data types: float
cols_obj_dencounty = ['Population', 'Area', 'Density']

df_popdencounty[cols_obj_dencounty] = df_popdencounty[cols_obj_dencounty].astype(float)

In [110]:
df_popdencounty.dtypes

County         object
State          object
FIPS Code      object
Population    float64
Area          float64
Density       float64
dtype: object

In [112]:
df_popdencounty.describe()

,Population,Area,Density
count,3.048000e+03,3048.000000,3048.000000
mean,1.022068e+05,1151.267060,283.357612
std,3.333259e+05,6208.437897,1996.890935
min,2.000000e+00,1.000000,0.000000
25%,9.844000e+03,434.000000,14.000000
50%,2.426000e+04,622.000000,41.000000
75%,6.576650e+04,934.250000,112.250000
max,9.757179e+06,319852.000000,70915.000000


#### 3. Data wranging

#### Dropping unnecessary columns

#### Population density: State

In [117]:
df_popdenstate = df_popdenstate.drop(['GEO.id', 'GEO.id2'], axis=1) 	

In [119]:
df_popdenstate.head()

,GEO.display-label,Density per square mile of land area
0,Alabama,94.4
1,Alaska,1.2
2,Arizona,56.3
3,Arkansas,56.0
4,California,239.1


#### Population density: County

In [122]:
df_popdencounty = df_popdencounty.drop(['FIPS Code'], axis=1) 	

In [124]:
df_popdencounty.head()

,County,State,Population,Area,Density
0,Autauga County,Alabama,61464.0,594.0,103.0
1,Baldwin County,Alabama,261608.0,1589.0,165.0
2,Barbour County,Alabama,24358.0,884.0,28.0
3,Bibb County,Alabama,22258.0,622.0,36.0
4,Blount County,Alabama,60163.0,644.0,93.0


#### Changing column names and county names

In [127]:
#Changing column names in df_popdenstate
df_popdenstate.rename(columns={'GEO.display-label': 'State',
                               'Density per square mile of land area': 'Pop_Density_sqmi'}, inplace=True)

In [129]:
df_popdenstate.head()

,State,Pop_Density_sqmi
0,Alabama,94.4
1,Alaska,1.2
2,Arizona,56.3
3,Arkansas,56.0
4,California,239.1


In [131]:
#Changing column names in df_popdencounty
df_popdencounty.rename(columns={'Density': 'Cty_Pop_Density_sqmi'}, inplace=True)

In [133]:
#Dropping 'County' from county names to standardize with the master df
df_popdencounty['County'] = df_popdencounty['County'].str.replace(' County', '', regex=False)

In [135]:
df_popdencounty.head()

,County,State,Population,Area,Cty_Pop_Density_sqmi
0,Autauga,Alabama,61464.0,594.0,103.0
1,Baldwin,Alabama,261608.0,1589.0,165.0
2,Barbour,Alabama,24358.0,884.0,28.0
3,Bibb,Alabama,22258.0,622.0,36.0
4,Blount,Alabama,60163.0,644.0,93.0


#### Merging datasets 

In [138]:
df_merged = pd.merge(df, df_gunindex, on='State', how='left')
df_merged = pd.merge(df_merged, df_popdenstate, on='State', how='left')
df_merged = pd.merge(df_merged, df_popdencounty, on='County', how='left')

In [140]:
df_merged.head()

,Incident_ID,Date,Year,Month,State_x,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population_x,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested,Category,Gun_Law_Strength,Gun_Violence_Rate,Pop_Density_sqmi,State_y,Population_y,Area,Cty_Pop_Density_sqmi
0,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Alabama,235969.0,784.0,301.0
1,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Illinois,20542.0,758.0,27.0
2,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Indiana,45654.0,411.0,111.0
3,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Iowa,11821.0,591.0,20.0
4,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Kentucky,50124.0,384.0,131.0


In [142]:
#Missing value check
df_merged.isnull().sum()


Incident_ID                  0
Date                         0
Year                         0
Month                        0
State_x                      0
City                         0
Victims_Killed               0
Victims_Injured              0
Suspects_Killed              0
Suspects_Injured             0
Suspects_Arrested            0
Year_Month_State             0
Handguns_Sold                0
Long_Guns_Sold               0
Total_Guns_Sold              0
State_ID                     0
State_y                      0
County                       0
Lat                          0
Long                         0
Population_x                 0
Shootings_County             0
Shootings_State              0
Total_Harmed_Victims         0
Any_Suspect_Arrested         0
Category                166016
Gun_Law_Strength        166016
Gun_Violence_Rate       166016
Pop_Density_sqmi         13869
State_y                   1579
Population_y              1579
Area                      1579
Cty_Pop_

In [144]:
df_merged['Population_x'].value_counts()

Population_x
0.0         75520
34688.0      9000
770383.0     7500
35626.0      6300
34804.0      5400
            ...  
233031.0        1
29003.0         1
282963.0        1
81824.0         1
93439.0         1
Name: count, Length: 5657, dtype: int64

In [146]:
df_merged[df_merged.isnull().any(axis=1)]

,Incident_ID,Date,Year,Month,State_x,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population_x,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested,Category,Gun_Law_Strength,Gun_Violence_Rate,Pop_Density_sqmi,State_y,Population_y,Area,Cty_Pop_Density_sqmi
0,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Alabama,235969.0,784.0,301.0
1,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Illinois,20542.0,758.0,27.0
2,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Indiana,45654.0,411.0,111.0
3,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Iowa,11821.0,591.0,20.0
4,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Kentucky,50124.0,384.0,131.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892627,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,IN,Indiana,Clark,38.4891,-85.7679,0.0,5,386,5,Yes,NaN,NaN,NaN,153.9,Washington,527269.0,11.0,47934.0
892628,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,IN,Indiana,Clark,38.4891,-85.7679,0.0,5,386,5,Yes,NaN,NaN,NaN,153.9,Wisconsin,34801.0,1209.0,29.0
892629,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,NE,Nebraska,Saunders,41.0946,-96.4332,0.0,3,386,5,Yes,NaN,NaN,NaN,153.9,Nebraska,23406.0,754.0,31.0
892630,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,AL,Alabama,Pickens,33.1369,-88.3008,0.0,3,386,5,Yes,NaN,NaN,NaN,153.9,Alabama,18508.0,881.0,21.0


In [148]:
#Duplicates check
dups_merged = df_merged.duplicated()

In [149]:
df_merged[df_merged.duplicated()]

,Incident_ID,Date,Year,Month,State_x,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population_x,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested,Category,Gun_Law_Strength,Gun_Violence_Rate,Pop_Density_sqmi,State_y,Population_y,Area,Cty_Pop_Density_sqmi
553985,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No,National leader,85,"13,5",231.1,Illinois,48716.0,716.0,68.0
553986,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No,National leader,85,"13,5",231.1,Indiana,35872.0,516.0,70.0
553987,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No,National leader,85,"13,5",231.1,Kentucky,29657.0,388.0,76.0
553988,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No,National leader,85,"13,5",231.1,Maine,40981.0,1142.0,36.0
553989,1582816,2019-12-29,2019,12,Illinois,Danville,0,5,0,0,0,2019-12-Illinois,28489.0,15802.0,44290.0,OH,Ohio,Knox,40.4473,-82.2612,49669.0,9,333,5,No,National leader,85,"13,5",231.1,Missouri,3740.0,506.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892605,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes,NaN,NaN,NaN,153.9,Kentucky,50124.0,384.0,131.0
892606,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes,NaN,NaN,NaN,153.9,Missouri,5926.0,501.0,12.0
892607,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes,NaN,NaN,NaN,153.9,Ohio,47952.0,409.0,117.0
892608,272016,2015-01-01,2015,1,Tennessee,Memphis,0,5,0,0,2,2015-1-Tennessee,25484.0,17575.0,43058.0,TN,Tennessee,Shelby,35.1087,-89.9663,937598.0,27,386,5,Yes,NaN,NaN,NaN,153.9,Tennessee,910530.0,755.0,1206.0


In [150]:
#Dropping duplicates
df_merged = df_merged.drop_duplicates()

In [152]:
df_merged[df_merged.duplicated()]

,Incident_ID,Date,Year,Month,State_x,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population_x,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested,Category,Gun_Law_Strength,Gun_Violence_Rate,Pop_Density_sqmi,State_y,Population_y,Area,Cty_Pop_Density_sqmi


In [153]:
#Dropping unnecessary columns

In [153]:
df_merged.dtypes

Incident_ID               int64
Date                     object
Year                      int64
Month                     int64
State_x                  object
City                     object
Victims_Killed            int64
Victims_Injured           int64
Suspects_Killed           int64
Suspects_Injured          int64
Suspects_Arrested         int64
Year_Month_State         object
Handguns_Sold           float64
Long_Guns_Sold          float64
Total_Guns_Sold         float64
State_ID                 object
State_y                  object
County                   object
Lat                     float64
Long                    float64
Population_x            float64
Shootings_County          int64
Shootings_State           int64
Total_Harmed_Victims      int64
Any_Suspect_Arrested     object
Category                 object
Gun_Law_Strength         object
Gun_Violence_Rate        object
Pop_Density_sqmi        float64
State_y                  object
Population_y            float64
Area    

In [156]:
df_merged = df_merged.drop(columns=['State_y'])

In [158]:
df_merged = df_merged.drop(['State_ID', 'Any_Suspect_Arrested', 'Population_y', 'Area'], axis=1) 	

In [162]:
df_merged.columns

Index(['Incident_ID', 'Date', 'Year', 'Month', 'State', 'City',
       'Victims_Killed', 'Victims_Injured', 'Suspects_Killed',
       'Suspects_Injured', 'Suspects_Arrested', 'Year_Month_State',
       'Handguns_Sold', 'Long_Guns_Sold', 'Total_Guns_Sold', 'State_ID',
       'State_y', 'County', 'Lat', 'Long', 'Population', 'Shootings_County',
       'Shootings_State', 'Total_Harmed_Victims', 'Any_Suspect_Arrested',
       'Category', 'Gun_Law_Strength', 'Gun_Violence_Rate',
       'St_Pop_Density_sqmi', 'State_y', 'Population_y', 'Area',
       'Cty_Pop_Density_sqmi'],
      dtype='object')

In [164]:
#Changing column names in df_popdencounty
df_merged.rename(columns={'State_x': 'State',
                            'Population_x': 'Population', 
                            'Pop_Density_sqmi': 'St_Pop_Density_sqmi',
                            }, inplace=True)

C:\Users\Alejandra\AppData\Local\Temp\ipykernel_19392\1092337948.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged.rename(columns={'State_x': 'State',


In [166]:
df_merged.head()

,Incident_ID,Date,Year,Month,State,City,Victims_Killed,Victims_Injured,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Year_Month_State,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,State_ID,State_y,County,Lat,Long,Population,Shootings_County,Shootings_State,Total_Harmed_Victims,Any_Suspect_Arrested,Category,Gun_Law_Strength,Gun_Violence_Rate,St_Pop_Density_sqmi,State_y,Population_y,Area,Cty_Pop_Density_sqmi
0,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Alabama,235969.0,784.0,301.0
1,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Illinois,20542.0,758.0,27.0
2,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Indiana,45654.0,411.0,111.0
3,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Iowa,11821.0,591.0,20.0
4,3181158,2025-04-09,2025,4,Tennessee,Memphis,1,5,0,0,0,2025-4-Tennessee,0.0,0.0,0.0,TN,Tennessee,Shelby,35.1087,-89.9663,0.0,3,30,6,No,NaN,NaN,NaN,153.9,Kentucky,50124.0,384.0,131.0


In [168]:
df_merged.columns

Index(['Incident_ID', 'Date', 'Year', 'Month', 'State', 'City',
       'Victims_Killed', 'Victims_Injured', 'Suspects_Killed',
       'Suspects_Injured', 'Suspects_Arrested', 'Year_Month_State',
       'Handguns_Sold', 'Long_Guns_Sold', 'Total_Guns_Sold', 'State_ID',
       'State_y', 'County', 'Lat', 'Long', 'Population', 'Shootings_County',
       'Shootings_State', 'Total_Harmed_Victims', 'Any_Suspect_Arrested',
       'Category', 'Gun_Law_Strength', 'Gun_Violence_Rate',
       'St_Pop_Density_sqmi', 'State_y', 'Population_y', 'Area',
       'Cty_Pop_Density_sqmi'],
      dtype='object')

In [172]:
#Reorganizing table order
new_order = ['Incident_ID', 'Date', 'Year', 'Month', 'Year_Month_State', 'State', 'County', 'City', 'Lat',
       'Long', 'Population', 'St_Pop_Density_sqmi', 'Cty_Pop_Density_sqmi', 'Victims_Killed', 'Victims_Injured', 'Total_Harmed_Victims', 'Suspects_Killed',
       'Suspects_Injured', 'Suspects_Arrested', 'Shootings_State', 'Shootings_County',
       'Handguns_Sold', 'Long_Guns_Sold', 'Total_Guns_Sold', 'Gun_Law_Strength',
       'Gun_Violence_Rate', 'Category']  
df_merged = df_merged[new_order]

In [174]:
df_merged

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Gun_Law_Strength,Gun_Violence_Rate,Category
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892627,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
892628,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
892629,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN
892630,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN


In [176]:
df_merged[df_merged.isnull().any(axis=1)]

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Gun_Law_Strength,Gun_Violence_Rate,Category
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892627,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
892628,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
892629,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN
892630,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN


In [178]:
#Correcting bad merge
df_merged = df_merged.drop(['Gun_Law_Strength', 'Gun_Violence_Rate', 'Category'], axis=1) 	

In [180]:
df_merged = pd.merge(df_merged, df_gunindex, on='State', how='left')

In [182]:
df_merged

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Category,Gun_Law_Strength,Gun_Violence_Rate
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602111,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
602112,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
602113,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN
602114,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN


In [184]:
df_merged[df_merged[['Category', 'Gun_Law_Strength', 'Gun_Violence_Rate']].isnull().any(axis=1)]

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Category,Gun_Law_Strength,Gun_Violence_Rate
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602111,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
602112,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN
602113,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN
602114,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN


In [186]:
df_merged['State'] = df_merged['State'].str.strip()
df_gunindex['State'] = df_gunindex['State'].str.strip()

In [188]:
print(df_merged['State'].dtype, df_gunindex['State'].dtype)

object object


In [190]:
df_merged.columns = df_merged.columns.str.strip()
df_gunindex.columns = df_gunindex.columns.str.strip()

In [192]:
print(set(df_merged['State']).difference(set(df_gunindex['State'])))

{'Tennessee', 'Wyoming', 'District Of Columbia', 'Mississippi'}


In [194]:
#Fixing typos
df_gunindex['State'] = df_gunindex['State'].replace({
    'Wyopming': 'Wyoming',
    'Mississipi': 'Mississippi',
    'Tennesee': 'Tennessee'
})

In [196]:
#Adding new entry
new_row = {
    'State': 'District of Columbia',
    'Category': 'none',
    'Gun_Law_Strength': 0,
    'Gun_Violence_Rate': 0
}

df_gunindex = pd.concat([df_gunindex, pd.DataFrame([new_row])], ignore_index=True)

In [198]:
#Retrying merge
df_merged = pd.merge(df_merged, df_gunindex, on='State', how='left')

In [200]:
df_merged[df_merged.isnull().any(axis=1)]

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Category_x,Gun_Law_Strength_x,Gun_Violence_Rate_x,Category_y,Gun_Law_Strength_y,Gun_Violence_Rate_y
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN,Weak systems,14,22
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN,Weak systems,14,22
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN,Weak systems,14,22
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN,Weak systems,14,22
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,NaN,NaN,NaN,Weak systems,14,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602111,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN,Weak systems,14,22
602112,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,NaN,NaN,NaN,Weak systems,14,22
602113,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN,Weak systems,14,22
602114,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,NaN,NaN,NaN,Weak systems,14,22


In [202]:
#Dropping bad columns
df_merged = df_merged.drop(['Category_x', 'Gun_Law_Strength_x', 'Gun_Violence_Rate_x'], axis=1) 	

In [204]:
#Changing column names 
df_merged.rename(columns={'Category_y': 'Category',
                            'Gun_Law_Strength_y': 'Gun_Law_Strength', 
                            'Gun_Violence_Rate_y': 'Gun_Violence_Rate'
                            }, inplace=True)

In [206]:
#Reorganizing table order
new_order = ['Incident_ID', 'Date', 'Year', 'Month', 'Year_Month_State', 'State', 'County', 'City', 'Lat',
       'Long', 'Population', 'St_Pop_Density_sqmi', 'Cty_Pop_Density_sqmi', 'Victims_Killed', 'Victims_Injured', 'Total_Harmed_Victims', 'Suspects_Killed',
       'Suspects_Injured', 'Suspects_Arrested', 'Shootings_State', 'Shootings_County',
       'Handguns_Sold', 'Long_Guns_Sold', 'Total_Guns_Sold', 'Gun_Law_Strength',
       'Gun_Violence_Rate', 'Category']  

df_merged = df_merged[new_order]

In [208]:
df_merged

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Gun_Law_Strength,Gun_Violence_Rate,Category
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602111,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,14,22,Weak systems
602112,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,14,22,Weak systems
602113,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,14,22,Weak systems
602114,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,14,22,Weak systems


In [210]:
dups_final = df_merged.duplicated()
dups_final.sum()

24549

In [211]:
df_merged_final = df_merged.drop_duplicates()

In [213]:
dups_final2 = df_merged_final.duplicated()
dups_final2.sum()

0

In [216]:
df_merged_final

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Gun_Law_Strength,Gun_Violence_Rate,Category
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602111,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,14,22,Weak systems
602112,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,14,22,Weak systems
602113,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,14,22,Weak systems
602114,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,14,22,Weak systems


In [218]:
df_merged_final[df_merged_final.isnull().any(axis=1)]

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Gun_Law_Strength,Gun_Violence_Rate,Category
31,3180141,2025-04-08,2025,4,2025-4-Virginia,Virginia,Fredericksburg,Fredericksburg,38.2992,-77.4872,0.0,202.6,NaN,3,3,6,0,1,4,17,1,0.0,0.0,0.0,49,"13,8",Making progress
146,3173826,2025-03-31,2025,3,2025-3-Texas,Texas,Dorado,San Antonio,18.4471,-66.2995,0.0,96.3,NaN,0,4,4,1,0,0,56,2,0.0,0.0,0.0,"13,5","14,9",Weak systems
147,3173826,2025-03-31,2025,3,2025-3-Texas,Texas,Quebradillas,San Antonio,18.4391,-66.9321,0.0,96.3,NaN,0,4,4,1,0,0,56,2,0.0,0.0,0.0,"13,5","14,9",Weak systems
148,3173826,2025-03-31,2025,3,2025-3-Texas,Texas,Aguadilla,San Antonio,18.4932,-67.0997,0.0,96.3,NaN,0,4,4,1,0,0,56,2,0.0,0.0,0.0,"13,5","14,9",Weak systems
151,3173872,2025-03-30,2025,3,2025-3-California,California,San Francisco,San Francisco,37.7558,-122.4449,0.0,239.1,NaN,0,4,4,0,0,0,14,1,0.0,0.0,0.0,"90,5",8,National leader
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601835,281039,2015-01-19,2015,1,2015-1-Texas,Texas,Dorado,San Antonio,18.4471,-66.2995,0.0,96.3,NaN,2,5,7,0,0,3,289,2,51105.0,42573.0,93678.0,"13,5","14,9",Weak systems
601836,281039,2015-01-19,2015,1,2015-1-Texas,Texas,Quebradillas,San Antonio,18.4391,-66.9321,0.0,96.3,NaN,2,5,7,0,0,3,289,2,51105.0,42573.0,93678.0,"13,5","14,9",Weak systems
601837,281039,2015-01-19,2015,1,2015-1-Texas,Texas,Aguadilla,San Antonio,18.4932,-67.0997,0.0,96.3,NaN,2,5,7,0,0,3,289,2,51105.0,42573.0,93678.0,"13,5","14,9",Weak systems
601893,298773,2015-01-13,2015,1,2015-1-Virginia,Virginia,Portsmouth,Portsmouth,36.8468,-76.3540,0.0,202.6,NaN,0,5,5,0,0,0,95,1,23125.0,15876.0,39002.0,49,"13,8",Making progress


In [220]:
pd.set_option('display.max_columns', None)

In [222]:
df_merged_final.isnull().sum()

Incident_ID                 0
Date                        0
Year                        0
Month                       0
Year_Month_State            0
State                       0
County                      0
City                        0
Lat                         0
Long                        0
Population                  0
St_Pop_Density_sqmi     13467
Cty_Pop_Density_sqmi     1493
Victims_Killed              0
Victims_Injured             0
Total_Harmed_Victims        0
Suspects_Killed             0
Suspects_Injured            0
Suspects_Arrested           0
Shootings_State             0
Shootings_County            0
Handguns_Sold               0
Long_Guns_Sold              0
Total_Guns_Sold             0
Gun_Law_Strength        13467
Gun_Violence_Rate       13467
Category                13467
dtype: int64

In [224]:
#Replacing NaN with 0: NaNs in this case are due to nonexistent records. 
df_merged_final = df_merged_final.fillna(0)


In [226]:
df_merged_final.isnull().sum()

Incident_ID             0
Date                    0
Year                    0
Month                   0
Year_Month_State        0
State                   0
County                  0
City                    0
Lat                     0
Long                    0
Population              0
St_Pop_Density_sqmi     0
Cty_Pop_Density_sqmi    0
Victims_Killed          0
Victims_Injured         0
Total_Harmed_Victims    0
Suspects_Killed         0
Suspects_Injured        0
Suspects_Arrested       0
Shootings_State         0
Shootings_County        0
Handguns_Sold           0
Long_Guns_Sold          0
Total_Guns_Sold         0
Gun_Law_Strength        0
Gun_Violence_Rate       0
Category                0
dtype: int64

In [228]:
df_final = df_merged_final

In [230]:
df_final

,Incident_ID,Date,Year,Month,Year_Month_State,State,County,City,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold,Gun_Law_Strength,Gun_Violence_Rate,Category
0,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,301.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
1,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,27.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
2,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,111.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
3,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,20.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
4,3181158,2025-04-09,2025,4,2025-4-Tennessee,Tennessee,Shelby,Memphis,35.1087,-89.9663,0.0,153.9,131.0,1,5,6,0,0,0,30,3,0.0,0.0,0.0,14,22,Weak systems
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602111,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,47934.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,14,22,Weak systems
602112,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Clark,Memphis,38.4891,-85.7679,0.0,153.9,29.0,0,5,5,0,0,2,386,5,25484.0,17575.0,43058.0,14,22,Weak systems
602113,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Saunders,Memphis,41.0946,-96.4332,0.0,153.9,31.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,14,22,Weak systems
602114,272016,2015-01-01,2015,1,2015-1-Tennessee,Tennessee,Pickens,Memphis,33.1369,-88.3008,0.0,153.9,21.0,0,5,5,0,0,2,386,3,25484.0,17575.0,43058.0,14,22,Weak systems


In [232]:
dup = df_final.duplicated()
dup.sum()

0

In [234]:
df_final.isnull().sum()

Incident_ID             0
Date                    0
Year                    0
Month                   0
Year_Month_State        0
State                   0
County                  0
City                    0
Lat                     0
Long                    0
Population              0
St_Pop_Density_sqmi     0
Cty_Pop_Density_sqmi    0
Victims_Killed          0
Victims_Injured         0
Total_Harmed_Victims    0
Suspects_Killed         0
Suspects_Injured        0
Suspects_Arrested       0
Shootings_State         0
Shootings_County        0
Handguns_Sold           0
Long_Guns_Sold          0
Total_Guns_Sold         0
Gun_Law_Strength        0
Gun_Violence_Rate       0
Category                0
dtype: int64

#### Exporting frame 

In [237]:
df_final.to_csv(os.path.join(path, '02_Data', 'Prepared', '150425_finalproject_2.csv'))

#### 4. Exploratory analysis

In [240]:
df_final.describe()

,Incident_ID,Year,Month,Lat,Long,Population,St_Pop_Density_sqmi,Cty_Pop_Density_sqmi,Victims_Killed,Victims_Injured,Total_Harmed_Victims,Suspects_Killed,Suspects_Injured,Suspects_Arrested,Shootings_State,Shootings_County,Handguns_Sold,Long_Guns_Sold,Total_Guns_Sold
count,5.775670e+05,577567.000000,577567.000000,577567.000000,577567.000000,5.775670e+05,577567.000000,577567.000000,577567.000000,577567.000000,577567.000000,577567.000000,577567.000000,577567.000000,577567.000000,577567.000000,577567.00000,577567.000000,577567.000000
mean,2.293309e+06,2021.809828,6.624535,38.520081,-89.442507,1.190317e+05,161.358477,278.341495,0.974309,4.158518,5.132828,0.071072,0.048349,0.719018,531.901327,116.764857,29717.49949,18222.764919,47940.245166
std,5.998540e+05,2.030511,3.020515,3.768586,9.886973,5.856961e+05,133.274546,1740.351146,1.231360,3.218048,3.399231,0.263158,0.235878,1.129136,328.231340,118.250978,24361.01782,12213.802803,35541.889313
min,2.720160e+05,2015.000000,1.000000,17.977800,-170.274300,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000,0.000000,0.000000
25%,1.793559e+06,2020.000000,4.000000,35.949000,-94.145100,1.094500e+04,87.100000,24.000000,0.000000,3.000000,4.000000,0.000000,0.000000,0.000000,267.000000,24.000000,11663.00000,10337.000000,22170.000000
50%,2.423331e+06,2022.000000,7.000000,39.044800,-87.686600,2.718200e+04,109.900000,55.000000,1.000000,4.000000,4.000000,0.000000,0.000000,0.000000,497.000000,72.000000,23205.00000,15336.000000,37650.000000
75%,2.737798e+06,2023.000000,9.000000,40.576600,-82.985500,6.434100e+04,202.600000,161.000000,1.000000,5.000000,5.000000,0.000000,0.000000,1.000000,712.000000,180.000000,36602.00000,22864.000000,60215.000000
max,3.181158e+06,2025.000000,12.000000,68.344500,-65.773300,3.129083e+07,1195.500000,70915.000000,60.000000,439.000000,499.000000,3.000000,5.000000,14.000000,1303.000000,483.000000,171600.00000,102546.000000,248724.000000


In [242]:
df_final.dtypes

Incident_ID               int64
Date                     object
Year                      int64
Month                     int64
Year_Month_State         object
State                    object
County                   object
City                     object
Lat                     float64
Long                    float64
Population              float64
St_Pop_Density_sqmi     float64
Cty_Pop_Density_sqmi    float64
Victims_Killed            int64
Victims_Injured           int64
Total_Harmed_Victims      int64
Suspects_Killed           int64
Suspects_Injured          int64
Suspects_Arrested         int64
Shootings_State           int64
Shootings_County          int64
Handguns_Sold           float64
Long_Guns_Sold          float64
Total_Guns_Sold         float64
Gun_Law_Strength         object
Gun_Violence_Rate        object
Category                 object
dtype: object

#### Splitting notebook 

Next section is in a different file